<a href="https://colab.research.google.com/github/Joracosu/03MAIR---Algoritmos-de-Optimizacion---2019/blob/master/AG1/Jos%C3%A9_Ram%C3%B3n_Conde_AG1_generico.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Actividad Guiada 1 de Algoritmos de Optimización

José Ramón Conde Suárez

https://colab.research.google.com/drive/1b0od-aP07ucKhmEqv2oQXfaofBG7L6km

## Ejercicio de las torres de Hanói

In [0]:
def torres (N, desde, hasta):
  if N==1:
    print('Lleva la ficha desde', str(desde),'hasta',str(hasta))
  else:
    torres(N-1, desde, 6-desde-hasta)
    print('Lleva la ficha desde', str(desde), 'hasta', str(hasta))
    torres(N-1, 6-desde-hasta, hasta)

# torres(3, 1, 3)

## Ejercicio de las n reinas

In [0]:
def reinas(N, solucion, etapa, j=0):
  for i in range(1, N + 1):
    solucion[etapa] = i
    if es_prometedora(solucion, etapa):
      if etapa == N - 1:
        j = j + 1
        print('Una solución es [',j,']:', solucion)
      else:
        j = reinas(N, solucion, etapa + 1, j)
    solucion[etapa] = 0
  return j

def es_prometedora(SOLUCION, etapa):
  for i in range(etapa + 1):
    if SOLUCION.count(SOLUCION[i]) > 1: return False
    for j in range(i + 1, etapa + 1):
      if abs(i-j) == abs(SOLUCION[i] - SOLUCION[j]): return False
  return True

# print('Total',reinas(8,([0,0,0,0,0,0,0,0]),0),'soluciones')


## Ejercicio de vecinos más cercanos


### Fuerza bruta
#### Todas las dimensiones

In [0]:
def puntos_mas_cercanos_bruto_multiD(lista):
  # se calcula un valor que seguro que es mayor que la distancia mínima de la lista
  auxiliar = 0
  for n in range(len(lista[0])-1):
    auxiliar += (max([k[n] for k in lista])-min([k[n] for k in lista]))**2
  minimo = (((auxiliar)**0.5)+1)*2

  for i in range(len(lista)-1): #para todos los valores de la lista menos el último
    for j in range(i+1,len(lista)): # comparar con el resto de valores de la lista
      p, q = lista[i], lista[j]

      # cálculo de la distancia euquidea independientemente de las dimensiones
      auxiliar = 0
      for n in range(len(lista[0])-1):
        auxiliar += (p[n]-q[n])**2
      dist = (auxiliar)**0.5
      
      if dist < minimo:
        # se reinicia la lista
        minimo = dist
        pareja = [[p, q]]
      elif dist == minimo:
        # se añade el punto a la lista
        pareja += [[p, q]]
  k = sorted(pareja)
  k = [k[i] for i in range(len(k)) if i == 0 or k[i] != k[i-1]]
  return minimo, k

### Divide y venceras
#### Todas las dimensiones

In [0]:
import random
from time import time
def puntos_mas_cercanos_divide_multiD(lista, dimension = -1):

  if dimension == -1: dimension = len(lista[0]) - 1

  if len(lista) <= 3:
    # para aplicar el método, lo primero es ordenar la lista
    lista = sorted(lista)
    # cuando la división resulta en una lista de tamaño 3 o menor
    min_0, pareja_0 = puntos_mas_cercanos_bruto_multiD(lista)
  else:
    # para aplicar el método, se ordena primero por la dimensión última
    if dimension == 1:
      lista = sorted(lista)
    else:
      lista = sorted(lista, key=lambda punto : punto[dimension - 1])
    # se divide la lista en dos partes
    largo = len(lista)
    lista_1 = lista[:int(largo/2)]
    lista_2 = lista[int(largo - largo/2):]
    
    # se aplica recurrentemente el algoritmo a cada división
    min_1, pareja_1 = puntos_mas_cercanos_divide_multiD(lista_1, dimension)
    min_2, pareja_2 = puntos_mas_cercanos_divide_multiD(lista_2, dimension)

    # Estudio de la frontera:
    #    Al tratarse de tres dimensiones, hay que estudiar TODOS los puntos que se
    #    encuentran a una distancia en z menor que la distancia mínima encontrada
    #    entre las dos listas. Para ello se crea una lista que contenga dichos 
    #    puntos independientemente de su X y de su Y y se calcula su distancia mínima por 
    #    divide y venceras en 2D
    if dimension == 1:
      p, q = lista_1[len(lista_1)-1], lista_2[0]
      min_3, pareja_3 = puntos_mas_cercanos_bruto_multiD([p, q])
    else:
      p_medio = lista[int(largo/2)-1]
      dm = min(min_1, min_2)
      lista = np.array(lista)
      # print(dimension)
      lista_3 = [tuple(p) for p in list(lista[[abs(i[dimension - 1] - p_medio[dimension - 1]) <= dm for i in lista]])]
      if len(lista_3)>1: min_3, pareja_3 = puntos_mas_cercanos_divide_multiD(lista_3, dimension - 1)
      else: min_3 = max(min_1, min_2) + 1  # con esto se declara un valor para min_3 pero se decarta

    # se almacenan todas las parejas de puntos de la distancia mínima
    # min_0 = min(min_1, min_2, min_3)
    if min_1 == min_2:
      if min_1 == min_3:
        min_0, pareja_0 = min_1, pareja_1 + pareja_3 + pareja_2
      elif min_1 < min_3:
        min_0, pareja_0 = min_1, pareja_1 + pareja_2
      else:
        min_0, pareja_0 = min_3, pareja_3
    elif min_1 > min_2:
      if min_2 == min_3:
        min_0, pareja_0 = min_2, pareja_3 + pareja_2
      elif min_2 < min_3:
        min_0, pareja_0 = min_2, pareja_2
      else:
        min_0, pareja_0 = min_3, pareja_3
    else:
      if min_1 == min_3:
        min_0, pareja_0 = min_1, pareja_1 + pareja_3
      elif min_1 < min_3:
        min_0, pareja_0 = min_1, pareja_1
      else:
        min_0, pareja_0 = min_3, pareja_3

  # finalmente se eliminan puntos duplicados
  k = sorted(pareja_0)
  k = [k[i] for i in range(len(k)) if i == 0 or k[i] != k[i-1]]
  return min_0, k

### Aplicación a listas

In [506]:
print('Dimensión 1')
precision = 100000000

lista_de_numeros = [(random.randrange(0,precision),[x+1]) for x in range(3000)]
lista = sorted(lista_de_numeros)

print('Fuerza bruta')
tiempo_1 = time()
min_a, puntos_a = puntos_mas_cercanos_bruto_multiD(lista)
tiempo_2 = time()
print(min_a, puntos_a)
print (tiempo_2 - tiempo_1)

print('\nDivide y vencerás')
tiempo_1 = time()
min_b, puntos_b = puntos_mas_cercanos_divide_multiD(lista)
tiempo_2 = time()
print(min_b, puntos_b)
print (tiempo_2 - tiempo_1)
print('---------------------------------------------------')

print('Dimensión 2')
precision = 20000

lista_de_numeros = [(random.randrange(0,precision),
                     random.randrange(0,precision),[x+1]) for x in range(2500)]
lista = sorted(lista_de_numeros)

print('Fuerza bruta')
tiempo_1 = time()
min_a, puntos_a = puntos_mas_cercanos_bruto_multiD(lista)
tiempo_2 = time()
print(min_a, puntos_a)
print (tiempo_2 - tiempo_1)

print('\nDivide y vencerás')
tiempo_1 = time()
min_b, puntos_b = puntos_mas_cercanos_divide_multiD(lista)
tiempo_2 = time()
print(min_b, puntos_b)
print (tiempo_2 - tiempo_1)
print('---------------------------------------------------')


print('Dimensión 3')
precision = 2000

lista_de_numeros = [(random.randrange(0,precision),
                     random.randrange(0,precision),
                     random.randrange(0,precision),[x+1]) for x in range(2000)]
lista = sorted(lista_de_numeros)

print('Fuerza bruta')
tiempo_1 = time()
min_a, puntos_a = puntos_mas_cercanos_bruto_multiD(lista)
tiempo_2 = time()
print(min_a, puntos_a)
print (tiempo_2 - tiempo_1)

print('\nDivide y vencerás')
tiempo_1 = time()
min_b, puntos_b = puntos_mas_cercanos_divide_multiD(lista)
tiempo_2 = time()
print(min_b, puntos_b)
print (tiempo_2 - tiempo_1)
print('---------------------------------------------------')

print('Dimensión 4')
precision = 2000

lista_de_numeros = [(random.randrange(0,precision),
                     random.randrange(0,precision),
                     random.randrange(0,precision),
                     random.randrange(0,precision),[x+1]) for x in range(1700)]
lista = sorted(lista_de_numeros)

print('Fuerza bruta')
tiempo_1 = time()
min_a, puntos_a = puntos_mas_cercanos_bruto_multiD(lista)
tiempo_2 = time()
print(min_a, puntos_a)
print (tiempo_2 - tiempo_1)

print('\nDivide y vencerás')
tiempo_1 = time()
min_b, puntos_b = puntos_mas_cercanos_divide_multiD(lista)
tiempo_2 = time()
print(min_b, puntos_b)
print (tiempo_2 - tiempo_1)
print('---------------------------------------------------')

print('Dimensión 5')
precision = 2000

lista_de_numeros = [(random.randrange(0,precision),
                     random.randrange(0,precision),
                     random.randrange(0,precision),
                     random.randrange(0,precision),
                     random.randrange(0,precision),[x+1]) for x in range(1400)]
lista = sorted(lista_de_numeros)

print('Fuerza bruta')
tiempo_1 = time()
min_a, puntos_a = puntos_mas_cercanos_bruto_multiD(lista)
tiempo_2 = time()
print(min_a, puntos_a)
print (tiempo_2 - tiempo_1)

print('\nDivide y vencerás')
tiempo_1 = time()
min_b, puntos_b = puntos_mas_cercanos_divide_multiD(lista)
tiempo_2 = time()
print(min_b, puntos_b)
print (tiempo_2 - tiempo_1)
print('---------------------------------------------------')

# a partir de la sexta dimensión, el algoritmo de Divide y Vencerás se vuelve más lento que el de fuerza bruta
print('Dimensión 6')
precision = 2000

lista_de_numeros = [(random.randrange(0,precision),
                     random.randrange(0,precision),
                     random.randrange(0,precision),
                     random.randrange(0,precision),
                     random.randrange(0,precision),
                     random.randrange(0,precision),[x+1]) for x in range(1200)]
lista = sorted(lista_de_numeros)

print('Fuerza bruta')
tiempo_1 = time()
min_a, puntos_a = puntos_mas_cercanos_bruto_multiD(lista)
tiempo_2 = time()
print(min_a, puntos_a)
print (tiempo_2 - tiempo_1)

print('\nDivide y vencerás')
tiempo_1 = time()
min_b, puntos_b = puntos_mas_cercanos_divide_multiD(lista)
tiempo_2 = time()
print(min_b, puntos_b)
print (tiempo_2 - tiempo_1)
print('---------------------------------------------------')


Dimensión 1
Fuerza bruta
19.0 [[(39558279, [554]), (39558298, [195])]]
3.4893136024475098

Divide y vencerás
19.0 [[(39558279, [554]), (39558298, [195])]]
0.01481938362121582
---------------------------------------------------
Dimensión 2
Fuerza bruta
1.0 [[(16761, 9880, [327]), (16761, 9881, [715])]]
3.1981658935546875

Divide y vencerás
1.0 [[(16761, 9880, [327]), (16761, 9881, [715])]]
0.09903860092163086
---------------------------------------------------
Dimensión 3
Fuerza bruta
10.488088481701515 [[(412, 1983, 1340, [1158]), (415, 1982, 1350, [557])]]
2.6261003017425537

Divide y vencerás
10.488088481701515 [[(412, 1983, 1340, [1158]), (415, 1982, 1350, [557])]]
0.2646162509918213
---------------------------------------------------
Dimensión 4
Fuerza bruta
24.372115213907882 [[(1245, 841, 1746, 1788, [1428]), (1262, 844, 1736, 1774, [66])]]
2.285400629043579

Divide y vencerás
24.372115213907882 [[(1245, 841, 1746, 1788, [1428]), (1262, 844, 1736, 1774, [66])]]
0.533806324005127
